Model Building

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import subplots, figure, show
from matplotlib import pyplot, pylab
from sklearn.model_selection import GridSearchCV

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn import metrics
from sklearn.svm import SVC

import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
df=pd.read_csv("MCI_2014_to_2019.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206435 entries, 0 to 206434
Data columns (total 29 columns):
X                      206435 non-null float64
Y                      206435 non-null float64
Index_                 206435 non-null int64
event_unique_id        206435 non-null object
occurrencedate         206435 non-null object
reporteddate           206435 non-null object
premisetype            206435 non-null object
ucr_code               206435 non-null int64
ucr_ext                206435 non-null int64
offence                206435 non-null object
reportedyear           206435 non-null int64
reportedmonth          206435 non-null object
reportedday            206435 non-null int64
reporteddayofyear      206435 non-null int64
reporteddayofweek      206435 non-null object
reportedhour           206435 non-null int64
occurrenceyear         206376 non-null float64
occurrencemonth        206376 non-null object
occurrenceday          206376 non-null float64
occurrencedayofyea

In [7]:
#df=df.drop(['event_unique_id','reporteddate','reportedyear','reportedmonth','reportedday','reporteddayofyear','reporteddayofweek','reportedhour'],axis=1)

In [8]:
#df=df.drop(['Index_'],axis=1)

In [5]:
df.head()

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Long,Lat,ObjectId
0,-79.405228,43.656982,7801,GO-20152165447,2015-12-18T03:58:00.000Z,2015-12-18T03:59:00.000Z,Commercial,1430,100,Assault,...,352.0,Friday,3,Assault,D14,79,University (79),-79.405228,43.656982,7001
1,-79.307907,43.778732,7802,GO-20151417245,2015-08-15T21:45:00.000Z,2015-08-17T22:11:00.000Z,Commercial,1430,100,Assault,...,227.0,Saturday,21,Assault,D42,118,Tam O'Shanter-Sullivan (118),-79.307907,43.778732,7002
2,-79.225029,43.765942,7803,GO-20151421107,2015-08-16T16:00:00.000Z,2015-08-18T14:40:00.000Z,Apartment,2120,200,B&E,...,228.0,Sunday,16,Break and Enter,D43,137,Woburn (137),-79.225029,43.765942,7003
3,-79.140823,43.778648,7804,GO-20152167714,2015-11-26T13:00:00.000Z,2015-12-18T13:38:00.000Z,Other,2120,200,B&E,...,330.0,Thursday,13,Break and Enter,D43,133,Centennial Scarborough (133),-79.140823,43.778648,7004
4,-79.288361,43.691235,7805,GO-20152169954,2015-12-18T19:50:00.000Z,2015-12-18T19:55:00.000Z,Commercial,1430,100,Assault,...,352.0,Friday,19,Assault,D55,61,Taylor-Massey (61),-79.288361,43.691235,7005


In [6]:
df = pd.read_csv('MCI_2014_to_2019.csv',sep=',') 
df = df.drop(df[df['occurrenceyear'] < 2014].index)
data_org = df.iloc[:,[22,6,17,20,21,25]]
data_org = data_org.dropna()
data_org.head()

,MCI,premisetype,occurrencemonth,occurrencedayofweek,occurrencehour,Neighbourhood
0,Assault,Commercial,December,Friday,3,University (79)
1,Assault,Commercial,August,Saturday,21,Tam O'Shanter-Sullivan (118)
2,Break and Enter,Apartment,August,Sunday,16,Woburn (137)
3,Break and Enter,Other,November,Thursday,13,Centennial Scarborough (133)
4,Assault,Commercial,December,Friday,19,Taylor-Massey (61)


In [9]:
data = data_org.copy()
# Get one hot encoding of columns premisetype
one_hot = pd.get_dummies(data['premisetype'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencemonth
one_hot = pd.get_dummies(data['occurrencemonth'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencedayofweek
one_hot = pd.get_dummies(data['occurrencedayofweek'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencehour
one_hot = pd.get_dummies(data['occurrencehour'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns Neighbourhood
one_hot = pd.get_dummies(data['Neighbourhood'])
# Join the encoded df
data = data.join(one_hot)

# Set column Neighbourhood as target
data = data.drop('premisetype',axis = 1)
data = data.drop('occurrencemonth',axis = 1)
data = data.drop('Neighbourhood',axis = 1)
data = data.drop('occurrencedayofweek',axis = 1)
data = data.drop('occurrencehour',axis = 1)

In [12]:
# Target selection
y = data['MCI']
data = data.drop('MCI',axis = 1)
# Features selection
X = data[data.columns[:]]  

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [29]:
clf = RandomForestClassifier(n_estimators=50, max_depth = 25, criterion = 'gini')

#Train the model using the training sets.
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
y_training = clf.predict(X_train)

In [30]:
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred), "\n")

Testing Accuracy: 60.16 %
Training Accuracy: 63.77 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.60      0.93      0.73     33148
     Auto Theft       0.67      0.13      0.22      7154
Break and Enter       0.61      0.40      0.48     12822
        Robbery       0.69      0.01      0.03      6487
     Theft Over       0.33      0.00      0.00      1986

       accuracy                           0.60     61597
      macro avg       0.58      0.30      0.29     61597
   weighted avg       0.61      0.60      0.52     61597
 



In [90]:


# hyperparameter tuning
param_grid = {   'n_estimators': [20, 30, 50], 
                 'max_depth': [10, 15, 20, 25],
             }

grid_clf = GridSearchCV(clf, param_grid, cv=10)
grid_clf.fit(X_train, y_train)
grid_clf.best_params_



{'max_depth': 25, 'n_estimators': 50}

In [70]:
#Building a Random Forest after tuning

clf_rf = RandomForestClassifier(n_estimators=80, max_depth = 20, criterion = 'gini')

#Train the model using the training sets.
clf_rf.fit(X_train,y_train)

y_pred_kn = clf_rf.predict(X_test)
y_training_kn = clf_rf.predict(X_train)

In [71]:
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred_kn) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training_kn)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred_kn), "\n")

Testing Accuracy: 59.29 %
Training Accuracy: 61.16 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.59      0.94      0.72     33148
     Auto Theft       0.67      0.10      0.18      7154
Break and Enter       0.61      0.36      0.45     12822
        Robbery       0.58      0.00      0.00      6487
     Theft Over       0.00      0.00      0.00      1986

       accuracy                           0.59     61597
      macro avg       0.49      0.28      0.27     61597
   weighted avg       0.58      0.59      0.51     61597
 



In [72]:
from pprint import pprint
print('Parameters currently in use:\n')
pprint(clf_rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 20,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 80,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [ ]:
#Logistic Regression

In [35]:
from sklearn.linear_model import LogisticRegression

In [41]:
logisticRegr = LogisticRegression()
clf_log=logisticRegr.fit(X_train, y_train)

In [43]:
#score = logisticRegr.score(X_test, y_test)
#y_pred_log = clf_log.predict(X_test)
#y_training_log = clf.predict(X_train)

In [44]:
#print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred_log) * 100, 2), "%")
#print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training_log)* 100, 2), "%\n")
#print( 'Classification report: \n', metrics.classification_report(y_test, y_pred_log), "\n")

Testing Accuracy: 58.69 %
Training Accuracy: 61.38 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.61      0.89      0.72     33148
     Auto Theft       0.49      0.20      0.29      7154
Break and Enter       0.51      0.39      0.44     12822
        Robbery       0.45      0.01      0.02      6487
     Theft Over       0.00      0.00      0.00      1986

       accuracy                           0.59     61597
      macro avg       0.41      0.30      0.29     61597
   weighted avg       0.54      0.59      0.52     61597
 



In [ ]:
#kmeans Classifier

In [91]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#Knn classifier! too slow to load
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(X_train, y_train)
ypred_knn = clf.predict(X_train)
acc = accuracy_score(y_true=y_train, y_pred=ypred_knn)
print('Acc: {:.4f}'.format(acc))

In [ ]:
#Going Bac to the originl dataset for more analysis

In [47]:
data_org.head()

,MCI,premisetype,occurrencemonth,occurrencedayofweek,occurrencehour,Neighbourhood
0,Assault,Commercial,December,Friday,3,University (79)
1,Assault,Commercial,August,Saturday,21,Tam O'Shanter-Sullivan (118)
2,Break and Enter,Apartment,August,Sunday,16,Woburn (137)
3,Break and Enter,Other,November,Thursday,13,Centennial Scarborough (133)
4,Assault,Commercial,December,Friday,19,Taylor-Massey (61)


In [58]:
data_org['MCI'].unique()

array(['Assault', 'Break and Enter', 'Robbery', 'Theft Over',
       'Auto Theft'], dtype=object)

In [13]:
data_org.head()

,MCI,premisetype,occurrencemonth,occurrencedayofweek,occurrencehour,Neighbourhood
0,Assault,Commercial,December,Friday,3,University (79)
1,Assault,Commercial,August,Saturday,21,Tam O'Shanter-Sullivan (118)
2,Break and Enter,Apartment,August,Sunday,16,Woburn (137)
3,Break and Enter,Other,November,Thursday,13,Centennial Scarborough (133)
4,Assault,Commercial,December,Friday,19,Taylor-Massey (61)


In [ ]:
#SCALING

In [12]:
data_org.head()

,MCI,premisetype,occurrencemonth,occurrencedayofweek,occurrencehour,Neighbourhood
0,Assault,Commercial,December,Friday,3,University (79)
1,Assault,Commercial,August,Saturday,21,Tam O'Shanter-Sullivan (118)
2,Break and Enter,Apartment,August,Sunday,16,Woburn (137)
3,Break and Enter,Other,November,Thursday,13,Centennial Scarborough (133)
4,Assault,Commercial,December,Friday,19,Taylor-Massey (61)


In [16]:
y=data_org['MCI']

In [21]:
data_org_=data_org.drop(['MCI'],axis=1)

In [22]:
data_org_.head()

,premisetype,occurrencemonth,occurrencedayofweek,occurrencehour,Neighbourhood
0,Commercial,December,Friday,3,University (79)
1,Commercial,August,Saturday,21,Tam O'Shanter-Sullivan (118)
2,Apartment,August,Sunday,16,Woburn (137)
3,Other,November,Thursday,13,Centennial Scarborough (133)
4,Commercial,December,Friday,19,Taylor-Massey (61)


In [23]:
data_org_=pd.get_dummies(data_org_)

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
print(scaler.fit(data_org_))

MinMaxScaler(copy=True, feature_range=(0, 1))


In [25]:
data_org_.head()

,occurrencehour,premisetype_Apartment,premisetype_Commercial,premisetype_House,premisetype_Other,premisetype_Outside,occurrencemonth_April,occurrencemonth_August,occurrencemonth_December,occurrencemonth_February,...,Neighbourhood_Willowdale West (37),Neighbourhood_Willowridge-Martingrove-Richview (7),Neighbourhood_Woburn (137),Neighbourhood_Woodbine Corridor (64),Neighbourhood_Woodbine-Lumsden (60),Neighbourhood_Wychwood (94),Neighbourhood_Yonge-Eglinton (100),Neighbourhood_Yonge-St.Clair (97),Neighbourhood_York University Heights (27),Neighbourhood_Yorkdale-Glen Park (31)
0,3,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,21,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,13,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,19,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Target selection
#y = data['MCI']
#data = data.drop('MCI',axis = 1)
# Features selection
X = data_org_[data_org_.columns[:]]  

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [28]:
clf = RandomForestClassifier(n_estimators=50, max_depth = 25, criterion = 'gini')

#Train the model using the training sets.
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
y_training = clf.predict(X_train)

In [29]:
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred), "\n")

Testing Accuracy: 60.23 %
Training Accuracy: 66.19 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.61      0.91      0.73     33218
     Auto Theft       0.57      0.15      0.23      7038
Break and Enter       0.59      0.43      0.50     12881
        Robbery       0.45      0.04      0.08      6446
     Theft Over       0.14      0.01      0.01      2014

       accuracy                           0.60     61597
      macro avg       0.47      0.31      0.31     61597
   weighted avg       0.57      0.60      0.53     61597
 



In [ ]:
#Creating models for the top 3 MCI

In [7]:
data_org.head()

,MCI,premisetype,occurrencemonth,occurrencedayofweek,occurrencehour,Neighbourhood
0,Assault,Commercial,December,Friday,3,University (79)
1,Assault,Commercial,August,Saturday,21,Tam O'Shanter-Sullivan (118)
2,Break and Enter,Apartment,August,Sunday,16,Woburn (137)
3,Break and Enter,Other,November,Thursday,13,Centennial Scarborough (133)
4,Assault,Commercial,December,Friday,19,Taylor-Massey (61)


In [13]:
new_mci=['Assault','Auto Theft','Break and Enter']
data_org1=data_org[data_org.MCI.isin(new_mci)]

                   

In [15]:
data_org1.describe

<bound method NDFrame.describe of                     MCI premisetype occurrencemonth occurrencedayofweek  \
0               Assault  Commercial        December          Friday       
1               Assault  Commercial          August          Saturday     
2       Break and Enter   Apartment          August          Sunday       
3       Break and Enter       Other        November          Thursday     
4               Assault  Commercial        December          Friday       
...                 ...         ...             ...                 ...   
206430       Auto Theft     Outside        February          Thursday     
206431       Auto Theft  Commercial        February          Thursday     
206432       Auto Theft       House         January          Wednesday    
206433       Auto Theft  Commercial        February          Thursday     
206434       Auto Theft       House        February          Thursday     

        occurrencehour                         Neighbourhood  
0 

In [17]:
data = data_org1.copy()
# Get one hot encoding of columns premisetype
one_hot = pd.get_dummies(data['premisetype'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencemonth
one_hot = pd.get_dummies(data['occurrencemonth'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencedayofweek
one_hot = pd.get_dummies(data['occurrencedayofweek'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns occurrencehour
one_hot = pd.get_dummies(data['occurrencehour'])
# Join the encoded df
data = data.join(one_hot)

# Get one hot encoding of columns Neighbourhood
one_hot = pd.get_dummies(data['Neighbourhood'])
# Join the encoded df
data = data.join(one_hot)

# Set column Neighbourhood as target
data = data.drop('premisetype',axis = 1)
data = data.drop('occurrencemonth',axis = 1)
data = data.drop('Neighbourhood',axis = 1)
data = data.drop('occurrencedayofweek',axis = 1)
data = data.drop('occurrencehour',axis = 1)

In [19]:
#Target selection
y = data['MCI']
data = data.drop('MCI',axis = 1)
# Features selection
X = data[data.columns[:]]  

In [20]:

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [21]:
clf = RandomForestClassifier(n_estimators=50, max_depth = 25, criterion = 'gini')


In [22]:
clf = RandomForestClassifier(n_estimators=50, max_depth = 25, criterion = 'gini')

#Train the model using the training sets.
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
y_training = clf.predict(X_train)

In [23]:
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred), "\n")

Testing Accuracy: 69.5 %
Training Accuracy: 73.2 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.71      0.92      0.80     33239
     Auto Theft       0.68      0.14      0.24      6920
Break and Enter       0.64      0.40      0.50     12987

       accuracy                           0.70     53146
      macro avg       0.67      0.49      0.51     53146
   weighted avg       0.69      0.70      0.65     53146
 



In [ ]:
#Logistic REGRESSION

In [24]:
from sklearn.linear_model import LogisticRegression

In [28]:
logisticRegr = LogisticRegression()
clf_log=logisticRegr.fit(X_train, y_train)

In [29]:
score = logisticRegr.score(X_test, y_test)
y_pred_log = clf_log.predict(X_test)
y_training_log = clf.predict(X_train)

In [30]:
print("Testing Accuracy:", round(metrics.accuracy_score(y_test, y_pred_log) * 100, 2), "%")
print("Training Accuracy:", round(metrics.accuracy_score(y_train, y_training_log)* 100, 2), "%\n")
print( 'Classification report: \n', metrics.classification_report(y_test, y_pred_log), "\n")

Testing Accuracy: 68.21 %
Training Accuracy: 73.2 %

Classification report: 
                  precision    recall  f1-score   support

        Assault       0.71      0.88      0.79     33239
     Auto Theft       0.57      0.27      0.37      6920
Break and Enter       0.58      0.40      0.47     12987

       accuracy                           0.68     53146
      macro avg       0.62      0.52      0.54     53146
   weighted avg       0.66      0.68      0.66     53146
 

